# prototype image generator class as base

```python
im_gen_obj = PatchImageGenerator()
patch_image = im_gen_obj.next_patch()
```

## *image_file_to_patches_directory_for_image_level.py*

### Fix and test:
* digipath_mltk.py & or digipath_toolkit.py
    * err-parameters checking

In [ ]:
import time
nb_start_time = time.time()

import os
import sys

from collections import OrderedDict
import argparse

import numpy as np
import pandas as pd
import yaml

from skimage.filters import threshold_otsu
from skimage.color import rgb2lab

from PIL import ImageDraw
from PIL import TiffImagePlugin as tip

import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import *

COMMON_THUMBNAIL_DIVISOR = 20

data_dir = '../../DigiPath_MLTK_data/Aperio'
file_type_list=['.svs', '.tif', '.tiff']
fs_od = get_file_size_ordered_dict(data_dir, file_type_list)
list_number = 0
for k, v in fs_od.items():
    print('%3i %30s: %i'%(list_number, k,v))
    list_number += 1

In [ ]:
class PatchImageGenerator_NB():
    """
    General case patch image generator for openslide Whole Slide Image file types

    Usage:  patch_image_generator = PatchImageGenerator(run_parameters)
            patch_dict = patch_image_generator.next_patch()

    Args:
        run_parameters:         (with these keys)
                                wsi_filename:           file name (with valid path)
                                patch_height:           patch size = (patch_width, patch_height)
                                patch_width:            patch size = (patch_width, patch_height)
                                thumbnail_divisor:      wsi_image full size divisor to create thumbnail image
                                patch_select_method:    'threshold_rgb2lab' or 'threshold_otsu'
                                threshold:              minimimum sum of thresholded image (default = 0)
    yields:
        patch_dict:             (with these keys)
                                patch_image:            PIL image of patch size
                                image_level_x:          column location in image level image
                                image_level_y:          row location in image level image
                                level_0_x:              column location in image (level 0)
                                level_0_y:              row location in image (level 0)
    """

    def __init__(self, run_parameters):
        self.os_obj = openslide.OpenSlide(run_parameters['wsi_filename'])
        self.image_level = run_parameters['image_level']
        self.patch_size = (run_parameters['patch_width'], run_parameters['patch_height'])
        _multi_ = self.os_obj.level_downsamples[self.image_level]
        self.image_level_loc_array = get_patch_location_array_for_image_level(run_parameters)
        self.level_0_location_array = [(int(p[0] * _multi_), int(p[1] * _multi_)) for p in self.image_level_loc_array]
        self._number_of_patches = len(self.image_level_loc_array)
        self._patch_number = -1

    def __del__(self):
        self.os_obj.close()

    def next_patch(self):
        self._patch_number += 1
        if self._patch_number < self._number_of_patches:
            patch_dict = {'patch_number': self._patch_number}
            patch_dict['image_level_x'] = self.image_level_loc_array[self._patch_number][1]
            patch_dict['image_level_y'] = self.image_level_loc_array[self._patch_number][0]
            patch_dict['level_0_x'] = self.level_0_location_array[self._patch_number][1]
            patch_dict['level_0_y'] = self.level_0_location_array[self._patch_number][0]

            location = (patch_dict['level_0_x'], patch_dict['level_0_y'])
            patch_dict['patch_image'] = self.os_obj.read_region(location, self.image_level, self.patch_size)

            return patch_dict

        else:
            raise StopIteration()


In [ ]:
data_dir = '../../DigiPath_MLTK_data/Aperio'
image_file_name = 'CMU-1.svs'

run_parameters = dict()

run_parameters['wsi_filename'] = os.path.join(data_dir, image_file_name)
print('Image File:\n', run_parameters['wsi_filename'])

run_parameters['thumbnail_divisor'] = COMMON_THUMBNAIL_DIVISOR
run_parameters['patch_select_method'] = 'threshold_otsu' # 'threshold_rgb2lab'
run_parameters['patch_height'] = 224
run_parameters['patch_width'] = 224
run_parameters['threshold'] = 0
run_parameters['image_level'] = 2

run_parameters['class_label'] = 'class_label_test_str'
run_parameters['output_dir'] = '../../run_dir/results'
            
patch_generator = PatchImageGenerator_NB(run_parameters)

In [ ]:
display(patch_generator.next_patch()['patch_image'])

In [ ]:
patch_generator = PatchImageGenerator_NB(run_parameters)
print()
patch_count = 0
while True:
    try:
        #         print(G.next_patch())
        patch_dict = patch_generator.next_patch()
        x = patch_dict['image_level_x']
        y = patch_dict['image_level_y']
        print('%8i x,\t%8i y\tsize = '%(x, y), patch_dict['patch_image'].size)
        # display(patch_generator.next_patch()['patch_image'])
        patch_count += 1
    except StopIteration:
        print('Thats all Folks')
        break
print(patch_dict['patch_number'], patch_count)

In [ ]:
help(patch_generator)